# Heterogenous diffusion: 1D

Simulating what might be happening in the protrusion w/ a spatially heterogenous diffusion constant (ie a nonhomogenous diffusion tensor).  I'll start this off with a simple 1D simulation.

*This is not intended to be a rigorous simulation.* It's merely a pilot to get a flavor for the behavior.

Define:
- **Positive curvature** = curving away from the original host cell's center

Assume:
- Fusogen is injected into the membrane and can freely diffuse (ie not physically tethered to anything)
    - Further assume that fusogen is injected into the very center of the protrusion tip
- Protrusion shape is torus inner quarter for base, cylinder for body, hemisphere for cap
- For practicality, fusogens that diffuse significantly beyond the base will just "fall off the edge of the world" and be dropped from the sim
    - We'll make the sim world end at 1 side length from the edge of "the base"
- For simplicity, the diffusion coefficient will be homegenous w/i the cap, the body, the base, and the outside buffer
- No collisions between fusogens.  Presumably the real fusogen machinery is injected at vanishingly low concentration relative to the membrane size, and so I'm really just interested in the probability of finding a fusogen at any given location over time, not their true dynamics.

Let's unfurl our 1D protrusion curve:
- The protrusion dimensions: ~4 um long, ~0.5 um cap radius of curvature, ~0.25 um base radius of curvature
    - $4 - 0.5 - 0.25 = 3.25 \mu m$ long sides x2 = length of outside buffer x2
    - $\frac{1}{2} 2 \pi 0.5 = 0.5 \pi \mu m$ half-circumference for the central cap
    - $\frac{1}{4} 2 \pi 0.25 = \frac{\pi}{8} \mu m$ quarter-circumference x2 for the base

Test:
- Fusogen has preference for positive curvature --> lower diffusion coefficient in positive curvature (higher binding energy w/ the membrane there); higher diffusion coefficient in negative curvature
- Fusogen has preference for negative curvature --> lower diffusion coefficient in positive curvature (higher binding energy w/ the membrane there); higher diffusion coefficient in negative curvature
- A stricly 1D test would keep the diffusion coefficient equal between the body and the buffer.  However, I want this to be a little more directly applicable to my real 2D situation, so I'm going to make them different in keeping with the different curvatures IRL

Like the diffusion model I did in Hernan's class, I'm going to model the diffusion as hopping between boxes with rate:
\begin{equation}
    \frac{\partial}{\partial t} N(i,t) = k_{i-1} N(i-1,t) + k_{i+1} N(i+1,t) - 2 k_{i} N(i,t)
\end{equation}
where the the first two terms represent flow into box i from its neighbors and the last term represents the flow out from box i to its neighbors

In [ ]:
%Code from class sans text
%NOTE: For some reason, this produces a wave packet of huge amplitude
    %when done with dt >= ~1E-5

%model params
D = 10; %um^2/s (diffusion constant)
tTot = 1E-3; %s
L = 2; %length of cell in um
N0 = 100; %# molecules

%sim params
dt = 1E-6; %s
dx = 0.01; %um
k = D/(dx^2); %jumping rate const; let be uniform for all cells
M = L/dx;
N = zeros(ceil(tTot/dt),M); %# molecs at each point at each time
N(1,M/2) = N0;
for t=2:tTot/dt %time iterations, start at 2nd row (1st already defined)
    N(t,1) = N(t-1,1) + N(t-1,1+1)*k*dt - N(t-1,1)*k*dt; %1st space cell
    
    for x=2:M-1 %middle space
       N(t,x) = N(t-1,x) + N(t-1,x-1)*k*dt + N(t-1,x+1)*k*dt... 
            -2*N(t-1,x)*k*dt;
    end
    
    N(t,M) = N(t-1,M) + N(t-1,M-1)*k*dt - N(t-1,M)*k*dt; %last space cell
end

cell = 1:M;
plot(cell,N(100,:)) %plot ditr at t=100
xlabel('Box #')
ylabel('# molecules')

hold on %plot next thing on same graph
plot(cell,N(400,:),'-r') %plot ditr at t=400 in red
    %choose this t since you see avg displacement 2x at 4x the time in
    %diffusion
legend('t = 100','t = 400')